In [2]:
import pandas as pd
import numpy as np
from pyvis.network import Network
from pybliometrics.scopus import AuthorSearch
from pybliometrics.scopus import AuthorRetrieval
from pprint import pprint
import gensim
from gensim import corpora
import itertools
import time
from sklearn.cluster import KMeans
import sys
sys.path.append("..")
from rcn_py import scopus

## Find the author_id by the name 

In [3]:

s = AuthorSearch('AUTHLAST(Kalverla) and AUTHFIRST(Peter)')  #todo also check Joris van Eijnatten

In [5]:
print(s)

Search 'AUTHLAST(Kalverla) and AUTHFIRST(Peter)' yielded 1 author as of 2023-02-28:
    Kalverla, Peter; AUTHOR_ID:57189467443 (8 document(s))


In [3]:
AuthorRetrieval(7004691405)

In [4]:
author_id = 57189467443
au = AuthorRetrieval(author_id)
print(au.self_link)



https://www.scopus.com/authid/detail.uri?partnerID=HzOxMe3b&authorId=57189467443&origin=inward


In [42]:
def nld_coauthor(author_id, depth, node_retrieved):
    au = AuthorRetrieval(author_id)
    docs = pd.DataFrame(au.get_documents())
    # Access to documents for the last five years
#     new_docs = docs[(docs.coverDate > '2018')]
    au_id = scopus.author_ids
    
    link = []
    all_node = []

    node_retrieved.append(author_id)
    
    for i in au_id:
        coau_id = i.split(";")
        coau_id = list(map(int, coau_id))
        new_coau_id = []
        for j in coau_id:
            aff = AuthorRetrieval(j).affiliation_current
            if AuthorRetrieval(j).affiliation_current:
                # Geo-filtering 
                # todo: all the affiliations 
                if AuthorRetrieval(j).affiliation_current[0][7] == 'nld':
                    if j not in all_node:
                        all_node.append(j)
                    new_coau_id.append(j)
                    
        sorted_new_coauid = list(map(int, new_coau_id))
        sorted_new_coauid.sort()
        link = link+list(itertools.combinations(sorted_new_coauid, 2))
        # Do recursion (increase depth of the network)
        if depth > 0:
            for j in sorted_new_coauid:
                if j not in node_retrieved:
                    nld_coauthor(j, depth-1)
    return all_node, link

In [7]:
node_retrieved = []
def all_coauthor(author_id, depth, node_retrieved):
    au = AuthorRetrieval(author_id)
    docs = pd.DataFrame(au.get_documents())
    # Access to documents for the last five years
    new_docs = docs[(docs.coverDate > '2018')]
    au_id = new_docs.author_ids
    
    link = []
    all_node = []

    node_retrieved.append(author_id)
    
    for i in au_id:
        coau_id = i.split(";")
        coau_id = list(map(int, coau_id))
        new_coau_id = []
        for j in coau_id:
            aff = AuthorRetrieval(j).affiliation_current
            if AuthorRetrieval(j).affiliation_current:
                # Geo-filtering 
                # todo: all the affiliations 
                # if AuthorRetrieval(j).affiliation_current[0][7] == 'nld':
                    if j not in all_node:
                        all_node.append(j)
                    new_coau_id.append(j)
                    
        sorted_new_coauid = list(map(int, new_coau_id))
        sorted_new_coauid.sort()
        link = link+list(itertools.combinations(sorted_new_coauid, 2))
        # Do recursion (increase depth of the network)
        if depth > 0:
            for j in sorted_new_coauid:
                if j not in node_retrieved:
                    nld_coauthor(j, depth-1)
    return all_node, link

In [8]:
node, link = all_coauthor(author_id, 0, node_retrieved)

In [21]:
node, link = nld_coauthor(author_id, 0, node_retrieved)

In [10]:
for i in node:
    print(AuthorRetrieval(i).given_name+' '+AuthorRetrieval(i).surname)

Rolf W. Hut
Niels Drost
Nick C. Van De Giesen
Ben J.C. van Werkhoven
Banafsheh Abdollahi
Jerom P.M. Aerts
Thomas Albers
Fakhereh Alidoost
Bouwe Andela
Jaro Camphuijsen
Yifat Dzigan
Ronald van Haren
Eric W.H. Hutton
Peter Kalverla
Maarten A.J. van Meersbergen
Gijs van den Oord
Inti Pelupessy
Stef Smeets
Stefan Verhoeven
Martine G. de Vos
Berend Weel
Mark Schelbergen
Roland Schmehl
Simon Watson
Albert A.M. Holtslag
Reinder J. Ronda
G. J. Steeneveld
James B. Duncan


In [12]:
def get_hindex(au_id):
    au = AuthorRetrieval(au_id)
    return au.h_index

In [22]:
sources = []
targets = []
weights = []

for i in link:
    sources.append(i[0])
    targets.append(i[1])
    weights.append(link.count(i))

In [25]:
# Pyvis network
N = Network(height=800, width="100%", bgcolor="#222222", font_color="white", select_menu=True)
N.toggle_hide_edges_on_drag(False)
N.barnes_hut()

edge_data = zip(sources, targets, weights)

for e in edge_data:
    src = e[0]
    dst = e[1]
    w = e[2]

    N.add_node(src, src, title=src, group = au_group[src])
    N.add_node(dst, dst, title=dst, group = au_group[dst])
    N.add_edge(src, dst, value=w)

neighbor_map = N.get_adj_list()

# add neighbor data to node hover data
for node in N.nodes:
    neighbors = []
    for neighbor_id in neighbor_map[node["id"]]:
        neigh = AuthorRetrieval(neighbor_id)
        neighbors.append(neigh.given_name+' '+neigh.surname)
    #node["title"] = " Neighbors: \n" + " \n".join(neighbors)
    node["title"] = "Link to the author’s API page:\n" + AuthorRetrieval(node["id"]).self_link
    node["value"] = get_hindex(node["id"])
    node["label"] = AuthorRetrieval(node["id"]).given_name+' '+AuthorRetrieval(node["id"]).surname
    # node["group"] = group[node["id"]]
    # node["font_size"] = get_hindex(node["id"])

N.show("peter.html")

In [14]:
au = AuthorRetrieval(author_id)
docs = pd.DataFrame(au.get_documents())

In [15]:
s.authors[0].eid.split('-')[-1]

'57189467443'

In [11]:
def filter_country(author_list, country_code):
    filtered_authors = []
    for i in author_list:
        aff_list = AuthorRetrieval(i).affiliation_current
        for aff_cur in aff_list:
            if aff_cur.country_code == country_code:
                filtered_authors.append(i)
                break
    return filtered_authors
        

In [23]:
author_lst = filter_country(node, 'nld')
print(len(author_lst))

27


In [24]:
def get_coauthor(author_first, author_last, depth):
    s = AuthorSearch('AUTHLAST('+author_last+') and AUTHFIRST('+author_first+')')
    author_id = s.authors[0].eid.split('-')[-1]
    node2 = []
    node, link = nld_coauthor(author_id, depth, node2)
    sources = []
    targets = []
    weights = []

    for i in link:
        sources.append(i[0])
        targets.append(i[1])
        weights.append(link.count(i))
    
    # Pyvis network
    N = Network(height=800, width="100%", bgcolor="#222222", font_color="white", select_menu=True)
    N.toggle_hide_edges_on_drag(False)
    N.barnes_hut()

    edge_data = zip(sources, targets, weights)

    for e in edge_data:
        src = e[0]
        dst = e[1]
        w = e[2]

        N.add_node(src, src, title=src)
        N.add_node(dst, dst, title=dst)
        N.add_edge(src, dst, value=w)

    neighbor_map = N.get_adj_list()

    # add neighbor data to node hover data
    for node in N.nodes:
        neighbors = []
        for neighbor_id in neighbor_map[node["id"]]:
            neigh = AuthorRetrieval(neighbor_id)
            neighbors.append(neigh.given_name+' '+neigh.surname)
        #node["title"] = " Neighbors: \n" + " \n".join(neighbors)
        node["title"] = "Link to the author’s API page:\n" + AuthorRetrieval(node["id"]).self_link
        node["value"] = get_hindex(node["id"])
        node["label"] = AuthorRetrieval(node["id"]).given_name+' '+AuthorRetrieval(node["id"]).surname
       

    N.show(author_last + ".html")


In [25]:
get_coauthor("Bart","Schilperoort",0) 

In [26]:
docs.description

0    Hutton et al. (2016) argued that computational...
1    Airborne wind energy (AWE) systems harness ene...
2    Offshore wind energy production is rapidly gro...
3    Ten years of ERA5 reanalysis data are combined...
4    Numerical weather prediction models play an im...
5    Uncertainty reduction in offshore wind systems...
6    In numerical models of geophysical fluid syste...
7    In the winter of 2012/13, the Katabatic Winds ...
Name: description, dtype: object

## Scopus search by subject code

In [1]:
from pybliometrics.scopus import SerialSearch

In [ ]:
s = SerialSearch(query={"subjCode": "2601"})

## Get Scopus Author ID from ORCID

This function solves the problem of overlap between the RSD and Scopus databases

In [4]:
import requests

In [42]:
orcid = "0000-0001-5371-5876"
url = "http://api.elsevier.com/content/search/author?query=ORCID%28"+orcid+"%29"
myAPIkey = "3d120b6ddb7d069272dfc2bc68af4028"

header = {'Accept' : 'application/json', 
          'X-ELS-APIKey' : myAPIkey}

resp = requests.get(url, headers=header)

results = resp.json()


In [54]:
preferred_name = results['search-results']['entry'][0]['preferred-name']
name  = preferred_name['surname'] + ' ' + preferred_name['initials']
print(name)

Casson F.J.


In [36]:
scopus_author_id = results['search-results']['entry'][0]['dc:identifier'].split(':')[-1]
print(scopus_author_id)

26025851000
